<a href="https://colab.research.google.com/github/rameshbabulakshmanan84/NLP-series/blob/master/Ber_ner_july_31_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#import pandas as pd
#path='/content/drive/My Drive/NLP/ner - custom.csv'
#data=pd.read_csv(path)
#data.tail(10)

In [3]:
import pandas as pd
path='/content/drive/My Drive/NLP/ner - custom-1.csv'
data=pd.read_csv(path,encoding='unicode_escape')
data.tail(10)

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,pos,prev-iob,prev-lemma,prev-pos,prev-prev-iob,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
14786,14786,expans,of,the,DT,lowercase,the,IN,lowercase,of,NN,O,further,JJ,O,ani,DT,lowercase,any,lowercase,further,665,lowercase,expansion,O
14787,14787,of,the,western,JJ,lowercase,western,DT,lowercase,the,IN,O,expans,NN,O,further,JJ,lowercase,further,lowercase,expansion,665,lowercase,of,O
14788,14788,the,western,allianc,NN,lowercase,alliance,JJ,lowercase,western,DT,O,of,IN,O,expans,NN,lowercase,expansion,lowercase,of,665,lowercase,the,O
14789,14789,western,allianc,on,IN,lowercase,on,NN,lowercase,alliance,JJ,O,the,DT,O,of,IN,lowercase,of,lowercase,the,665,lowercase,western,O
14790,14790,allianc,on,russia,NNP,capitalized,Russia,IN,lowercase,on,NN,O,western,JJ,O,the,DT,lowercase,the,lowercase,western,665,lowercase,alliance,O
14791,14791,on,russia,'s,POS,other,'s,NNP,capitalized,Russia,IN,O,allianc,NN,O,western,JJ,lowercase,western,lowercase,alliance,665,lowercase,on,O
14792,14792,russia,'s,border,NNS,lowercase,borders,POS,other,'s,NNP,O,on,IN,O,allianc,NN,lowercase,alliance,lowercase,on,665,capitalized,Russia,B-geo
14793,14793,'s,border,.,.,punct,.,NNS,lowercase,borders,POS,B-geo,russia,NNP,O,on,IN,lowercase,on,capitalized,Russia,665,other,'s,O
14794,14794,border,.,__end1__,__END1__,wildcard,__END1__,.,punct,.,NNS,O,'s,POS,B-geo,russia,NNP,capitalized,Russia,other,'s,665,lowercase,borders,O
14795,14795,.,__end1__,__end2__,__END2__,wildcard,__END2__,__END1__,wildcard,__END1__,.,O,border,NNS,O,'s,POS,other,'s,lowercase,borders,665,punct,.,O


In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [5]:
class SentenceGetter(object):

  def __init__(self,data):
    self.n_sent=1
    self.data=data
    self.empty=False
    agg_fun=lambda s: [(w,p,t) for w,p,t in zip(s["word"].values.tolist(),s["pos"].values.tolist(),s["tag"].values.tolist())]                                   
    self.grouped=self.data.groupby("sentence_idx").apply(agg_fun)
    self.sentences=[s for s in self.grouped]

  def get_next(self):
    try:
      s=self.grouped["Sentence:{}".format(self.n_sent)]
      self.n_sent += 1
      return s
    
    except:
      return None

In [6]:
#create an object of class sentenceGetter and pass dataframe 'data' as argument
getter=SentenceGetter(data)

In [7]:
for sentence in getter.sentences:
  print(sentence)
#getter.sentence form : list of lists of tuples with (words,pos,I/B/O-NER)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]
[('Families', 'NNS', 'O'), ('of', 'IN', 'O'), ('soldiers', 'NNS', 'O'), ('killed', 'VBN', 'O'), ('in', 'IN', 'O'), ('the', 'DT', 'O'), ('conflict', 'NN', 'O'), ('joined', 'VBD', 'O'), ('the', 'DT', 'O'), ('protesters', 'NNS', 'O'), ('who', 'WP', 'O'), ('carried', 'VBD', 'O'), ('banners', 'NNS', 'O'), ('with', 'IN', 'O'), ('such', 'JJ', 'O'), ('slogans', 'NNS', 'O'), ('as', 'IN', 'O'), ('"', '``', 'O'), ('Bush', 'NNP', 'B-per'), ('Number', 'NN', 'O

In [8]:
#retrieve the first sentence
#getter.sentence form : list of lists of tuples with (words,pos,I/B/O-NER)
#after that picke the first word in the sentence
#word[0] refers to all the words in sentences
#sentences[0] refers to the words in first sentences
sentences=[[word[0] for word in sentence] for sentence in getter.sentences ]
#sentences[0] refers to the first sentence words
sentences[0]

['Thousands',
 'of',
 'demonstrators',
 'have',
 'marched',
 'through',
 'London',
 'to',
 'protest',
 'the',
 'war',
 'in',
 'Iraq',
 'and',
 'demand',
 'the',
 'withdrawal',
 'of',
 'British',
 'troops',
 'from',
 'that',
 'country',
 '.']

In [9]:
 #s[0] : word,s[1] : pos , s[2]:ner
 labels=[[s[2] for s in sentence] for sentence in getter.sentences]
 #labels[[0]] : ner for first sentences
 print(labels[0])

['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [10]:
#map the tags with integers -> tags refer to ner
#data["tag"]:retrieves unique tags, set(data["tag"]) -> refers to set of distinct values
tag_values=list(set(data["tag"].values))
tag_values.append("PAD")
#create a dictionary of tag and values
tag2idx={t:i for i,t in enumerate(tag_values)}

In [11]:
print(tag2idx)
#Tags available :
#per-Person,geo:geogrpahy,gpe,art-Artist,org:Organization,tim-Time,

{'I-per': 0, 'I-org': 1, 'I-gpe': 2, 'I-geo': 3, 'B-eve': 4, 'B-per': 5, 'B-org': 6, 'I-art': 7, 'I-eve': 8, 'I-nat': 9, 'O': 10, 'B-geo': 11, 'I-tim': 12, 'B-art': 13, 'B-tim': 14, 'B-gpe': 15, 'B-nat': 16, 'PAD': 17}


In [12]:
!pip3 install transformers
#transformers in library from huggingface

     |████████████████████████████████| 778kB 4.7MB/s 
     |████████████████████████████████| 1.1MB 23.0MB/s 
     |████████████████████████████████| 890kB 31.7MB/s 
     |████████████████████████████████| 3.0MB 38.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ea4e39ac6136f43faf56f48cc5543ec19d02830132e32c9f50b760c353e6a35e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [13]:
#Apply the Bert
#use pytorch implementation of bert
import torch
#Dataset from a tensor or array or list or dict - TensorDataset
#DataLoader - represents a Python iterable over a dataset, with support for many datasets
#RandomSampler - A Sampler that returns random indices.
#SequentialSampler - A Sampler that returns indices sequentially.
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
#BertTokenizer -- constructs a tokenizer class - based on wordpiece tokenizer
#Bertconfg Config file for all parameters
from transformers import BertTokenizer,BertConfig
import logging
logging.basicConfig(level=logging.INFO)


In [14]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [26]:
MAX_LEN = 75
Batch_Size = 32

In [ ]:
#device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
#n_gpu=torch.cuda.device_count()

In [15]:
#BERTBASE -> Layers = 12,Hiddensize=768 dim, Attention A = 12,P=110M
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)


INFO:filelock:Lock 140037781339552 acquired on /root/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp56vrsxq5


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt in cache at /root/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:filelock:Lock 140037781339552 released on /root/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /root/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f2

In [ ]:
#split words into sub toekns
#We have to deal with the issue of splitting our token-level labels to related subtokens

In [16]:
sen=sentences[0]
print(sen)
labl=labels[0]
print(labl)


['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [17]:
for i,j in zip(sen,labl):
  tok_word=tokenizer.tokenize(i)
  print(i,tok_word)
  print(len(tok_word))

Thousands ['Thousands']
1
of ['of']
1
demonstrators ['demons', '##tra', '##tors']
3
have ['have']
1
marched ['marched']
1
through ['through']
1
London ['London']
1
to ['to']
1
protest ['protest']
1
the ['the']
1
war ['war']
1
in ['in']
1
Iraq ['Iraq']
1
and ['and']
1
demand ['demand']
1
the ['the']
1
withdrawal ['withdrawal']
1
of ['of']
1
British ['British']
1
troops ['troops']
1
from ['from']
1
that ['that']
1
country ['country']
1
. ['.']
1


In [18]:
def token_and_preserve_labels(sentence,text_labels):
  tokenized_sentence=[]
  labels=[]

  for word,label in zip(sentence,text_labels):
    #tokenize the word and count of # of subwords that word is toeknized into 
    #demonstrators ['demons', '##tra', '##tors']
    tokenized_word=tokenizer.tokenize(word)
    n_subwords=len(tokenized_word)

    # Add the tokenized word to the final tokenized word list
    tokenized_sentence.extend(tokenized_word)

    # Add the same label to the new list of labels `n_subwords` times
    labels.extend([label] * n_subwords)

  return tokenized_sentence,labels 


In [19]:
#sentence and corresponsing labels 
#sentence form : [list of words]
#labels form : #labels form [ list of ner tags]
tokenized_texts_and_labels=[token_and_preserve_labels(sent,labels) for sent,labels in zip(sentences,labels)]

In [20]:
print(tokenized_texts_and_labels)

[(['Thousands', 'of', 'demons', '##tra', '##tors', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']), (['Families', 'of', 'soldiers', 'killed', 'in', 'the', 'conflict', 'joined', 'the', 'protesters', 'who', 'carried', 'banner', '##s', 'with', 'such', 'slogan', '##s', 'as', '"', 'Bush', 'Number', 'One', 'Terror', '##ist', '"', 'and', '"', 'Stop', 'the', 'Bomb', '##ings', '.', '"'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['They', 'marched', 'from', 'the', 'Houses', 'of', 'Parliament', 'to', 'a', 'rally', 'in', 'Hyde', 'Park', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [21]:
# split text and labels
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]

labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [22]:
print(tokenized_texts)

[['Thousands', 'of', 'demons', '##tra', '##tors', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.'], ['Families', 'of', 'soldiers', 'killed', 'in', 'the', 'conflict', 'joined', 'the', 'protesters', 'who', 'carried', 'banner', '##s', 'with', 'such', 'slogan', '##s', 'as', '"', 'Bush', 'Number', 'One', 'Terror', '##ist', '"', 'and', '"', 'Stop', 'the', 'Bomb', '##ings', '.', '"'], ['They', 'marched', 'from', 'the', 'Houses', 'of', 'Parliament', 'to', 'a', 'rally', 'in', 'Hyde', 'Park', '.'], ['Police', 'put', 'the', 'number', 'of', 'march', '##ers', 'at', '10', ',', '000', 'while', 'organizers', 'claimed', 'it', 'was', '1', ',', '00', ',', '000', '.'], ['The', 'protest', 'comes', 'on', 'the', 'eve', 'of', 'the', 'annual', 'conference', 'of', 'Britain', "'", 's', 'ruling', 'Labor', 'Party', 'in', 'the', 'southern', 'English', 'seas', '##ide', 'resort', 'of', 

In [23]:
print(labels)

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'I-geo', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'B-org', 'I-org', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'B-geo', 'O'], ['O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'B-geo', 'O', 'O', 'B-geo', 'O'], ['O', 'B-org', 'I-org', 'I-org', 'I-org', 'O', '

Next, we cut and pad the token and label sequences to our desired length.

In [27]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [28]:
print(len(input_ids))

665


In [29]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [30]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [31]:
print(attention_masks)

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [32]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids,tags,random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2018, test_size=0.1)

In [33]:
#convert all tensor format
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [34]:
val_tags.shape

torch.Size([67, 75])

In [35]:
tr_inputs.shape

torch.Size([598, 75])

In [36]:
print(tr_tags)

tensor([[15, 10, 10,  ..., 17, 17, 17],
        [10,  6,  1,  ..., 17, 17, 17],
        [15, 10, 10,  ..., 17, 17, 17],
        ...,
        [10, 10, 10,  ..., 17, 17, 17],
        [10, 10, 10,  ..., 17, 17, 17],
        [15, 10, 10,  ..., 17, 17, 17]])


In [37]:
#train_data in the form - input tokens, attention masks for the corresponding tokens and the tags for the tokens
train_data = TensorDataset(tr_inputs,tr_masks,tr_tags)
#returns random indices from the data source
train_sampler = RandomSampler(train_data)
#dataloader combines the dataset and the sampler 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
#samples elements sequentially in the same order
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data,sampler=valid_sampler, batch_size=32)

Setup the Bert model for finetuning


In [39]:
import transformers
#bertfortokenclassification addsa classifier on top of bert model
from transformers import BertForTokenClassification,AdamW

In [40]:
transformers.__version__

'3.0.2'

In [41]:
model=BertForTokenClassification.from_pretrained("bert-base-cased",num_labels=len(tag2idx),output_attentions=False,output_hidden_states=False)
#bert-base-cased : 12 layer bert model cased
#trained bert model with a layer for classification
#output_attentions:  Whether the model returns attentions weights.
#output_hidden_states :  Whether the model returns all hidden-states.

INFO:filelock:Lock 140040270239392 acquired on /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmplkfkygdn


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json in cache at /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
INFO:filelock:Lock 140040270239392 released on /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391.lock
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df1

INFO:filelock:Lock 140040317152056 acquired on /root/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2.lock
INFO:transformers.file_utils:https://cdn.huggingface.co/bert-base-cased-pytorch_model.bin not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpijkrsrkn


INFO:transformers.file_utils:storing https://cdn.huggingface.co/bert-base-cased-pytorch_model.bin in cache at /root/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2
INFO:filelock:Lock 140040317152056 released on /root/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2.lock
INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/bert-base-cased-pytorch_model.bin from cache at /root/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2


- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#set up the optimizer
#ADAMW is the optimizer
#weight_Decay is regularization parameters
#if limited resources, train only the clasffier and not the weights with the labels


In [42]:
import torch
#specify GPU as device
if torch.cuda.is_available():
  device=torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [43]:
# Tell pytorch to run this model on the GPU.
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [44]:
#Print the different parameters
params=list(model.named_parameters())

#print
print("the model has {:} number of parameters".format(len(params)))

the model has 201 number of parameters


In [45]:
for p in params[0:5]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print("---First Transformer---")
for p in params[5:21]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
for p in params[190:200]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print("---output layer---")
for p in params[-4:]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)
---First Transformer---
bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (768,)
bert.encoder.layer.0.attention.output.LayerNorm.weight        (768,)
bert.encod

In [ ]:
#grab the training hyperparameters from within the stored model.

In [46]:
#grab the optimzer
optimizer=AdamW(model.parameters(),
               lr=2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5 
               eps=1e-8) #a very small number to prevent any division by zero in the implementation)

For the purposes of fine-tuning, the authors recommend choosing from the following values (from Appendix A.3 of the BERT paper):

Batch size: 16, 32
Learning rate (Adam): 5e-5, 3e-5, 2e-5
Number of epochs: 2, 3, 4

We also add a scheduler to linearly reduce the learning rate throughout the epochs.

> Indented block



In [47]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.


In [48]:
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).

In [49]:
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.

In [50]:
#Create a schedule with a learning rate that decreases linearly from the initial lr set in the optimizer to 0, after a warmup period during which it increases linearly from 0 to the initial lr set in the optimizer.

scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,#deafault value
    num_training_steps=total_steps)




Training Loop : Training/validation for each passes



In [51]:
#Helper function to check accuracy
import numpy as np

#Function to calculate the accuracy
#argmax : The indices of the max element axis. unravel_index : Convert a flat index into an index tuple. take_along_axis : Apply np.expand_dims(index_array, axis)
def flat_accuracy(preds,labels):
  pred_flat=np.argmax(preds,axis=1)
  labels_flat=labels.flatten()
  return np.sum(pred_flat==labels_flat)/len(labels_flat)



In [53]:
from sklearn.metrics import f1_score, accuracy_score


Helper function for formatting elapsed times as hh:mm:ss

In [54]:
import time,datetime

def format_time(elapsed):
  '''Takes a time in seconds and returns a string hh:mm:ss'''
  elapsed_rounded = int(round((elapsed)))

  return str(datetime.timedelta(seconds=elapsed_rounded))

Kick off the Training

In [55]:
import random
import numpy as np

In [56]:

# Set the seed value all over the place to make this reproducible.
seed_val=42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [57]:
# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

In [58]:
# Measure the total training time for the whole run.
total_t0=time.time()


In [65]:
#for ech epoch
#Run the training for the given epochs - typically between 2 and 4
#range 0 to 3

loss_values, validation_loss_values = [], []


for _ in trange(epochs,desc="EPOCH"):
  print("")
  #print ---Epoch 1/2 ----
  #print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  print('Training...')

  #Reset the total loss for this epoch
  total_train_loss=0
  
  #train the model
  model.train()

  for step,batch in enumerate(train_dataloader):
    #print(step)
    #print(batch)

    #add the batch to the GPU
    batch=tuple(t.to(device) for t in batch)
    b_input_ids,b_input_mask,b_labels=batch

  #always clear the gradients before doing a backward pass
    model.zero_grad()

   #perform a forward pass
    loss,logits=model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)
    
    #logits = logits.detach().cpu().numpy()
    #print("step number -", step)
    #print("score--",logits.shape)
    #outout shape in the form : 32*75*18 --32 batch size,75 --> sequence size, 18 --> Tags 
    #print("argmax--")
    #print("logits--",logits)
    #print("argmax--",np.argmax(logits,axis=1))
   #calculate the accumulating loss
   #`loss` is a Tensor containing a # single value; the `.item()` function just returns the Python value 
    total_train_loss+=loss.item()

   # Perform a backward pass to calculate the gradients.
   #compute gradient of loss w.r.t all the parameters in loss that have requires_grad = True and store them in parameter.grad attribute for every parameter.
    loss.backward()

   # Clip the norm of the gradients to 1.0.
   # This is to help prevent the "exploding gradients" problem
   #within a specific range (clip)
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

     
   # Update parameters and take a step using the computed gradient.
   # The optimizer dictates the "update rule"--how the parameters are
   # modified based on their gradients, the learning rate, etc.
   #updates all the parameters based on parameter.grad
    optimizer.step()

   # Update the learning rate.
    scheduler.step()
#Return to the epoch
   # Calculate the average loss over all of the batches.  
  avg_train_loss= total_train_loss/len(train_dataloader)
  print("Average train loss: {}".format(avg_train_loss))
  #append the loss for trainig
  loss_values.append(avg_train_loss)
  print(" Training is completed")
  print("Running Validation... ")

  #put the model in evaluation mode
  model.eval()

  # Tracking variables 
  total_eval_accuracy = 0
  total_eval_loss = 0
  nb_eval_steps = 0
  predictions , true_labels = [], []

  for batch in valid_dataloader:

    #add the batch to the GPU
    batch=tuple(t.to(device) for t in batch)
    b_input_ids,b_input_mask,b_labels=batch
   
    #print("validation labels:",b_labels)
     #always clear the gradients
     #in evaluation mode, you can't do backprog, to save time, dont compute the gradients
    with torch.no_grad():
      #perform a forward pass
      loss,logits=model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)

    logits=logits.detach().cpu().numpy()
    label_ids=b_labels.to('cpu').numpy()


   #calculate the accumulating loss
   #`loss` is a Tensor containing a # single value; the `.item()` function just returns the Python value 
    total_eval_loss+=loss.mean().item()
    predictions.extend([list(p) for p in np.argmax(logits,axis=2)])
    true_labels.extend(label_ids)

  eval_loss=total_eval_loss/len(valid_dataloader) 
  validation_loss_values.append(eval_loss)
  print("Validation loss: {}".format(eval_loss))

  pred_tags=[tag_values[p_i] for p,l in zip(predictions,true_labels) for p_i,l_i in zip(p,l) if tag_values[l_i] != "PAD"]

  valid_tags=[tag_values[l_i] for l in true_labels for l_i in l if tag_values[l_i]!="PAD"]

  print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
  print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags,average='macro')))

EPOCH:   0%|          | 0/4 [00:00<?, ?it/s]


Training...
Average train loss: 0.4021848408799422
 Training is completed
Running Validation... 


EPOCH:  25%|██▌       | 1/4 [00:08<00:25,  8.50s/it]

Validation loss: 0.2719571863611539
Validation Accuracy: 0.9035409035409036
Validation F1-Score: 0.24120343447169343

Training...
Average train loss: 0.40391416769278676
 Training is completed
Running Validation... 


EPOCH:  50%|█████     | 2/4 [00:17<00:17,  8.55s/it]

Validation loss: 0.2719571863611539
Validation Accuracy: 0.9035409035409036
Validation F1-Score: 0.24120343447169343

Training...
Average train loss: 0.40215045684262324
 Training is completed
Running Validation... 


EPOCH:  75%|███████▌  | 3/4 [00:26<00:08,  8.66s/it]

Validation loss: 0.2719571863611539
Validation Accuracy: 0.9035409035409036
Validation F1-Score: 0.24120343447169343

Training...
Average train loss: 0.4007921234557503
 Training is completed
Running Validation... 


EPOCH: 100%|██████████| 4/4 [00:35<00:00,  8.79s/it]

Validation loss: 0.2719571863611539
Validation Accuracy: 0.9035409035409036
Validation F1-Score: 0.24120343447169343


In [66]:
test_sentence = """
Mr. Trump’s tweets began just moments after a Fox News report by Mike Tobin, a 
reporter for the network, about protests in Minnesota and elsewhere. 
"""

In [67]:
#tokenize  the text
tokenized_sentence=tokenizer.encode(test_sentence)
#the output is sequence of ids (token to ids)

In [69]:
#conver to tensor format
input_ids = torch.tensor([tokenized_sentence]).cuda()

In [70]:
#run the sentence through the model
with torch.no_grad():
  output=model(input_ids)

In [74]:
label_indices=np.argmax(output[0].to('cpu').numpy(),axis=2)

In [75]:
print(label_indices)

[[10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
  10 10 10 10 10 10 10 10 10 10 10 10]]


In [79]:
#join the split tokens
tokens=tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])


In [77]:
print(tokens)

['[CLS]', 'Mr', '.', 'Trump', '’', 's', 't', '##weet', '##s', 'began', 'just', 'moments', 'after', 'a', 'Fox', 'News', 'report', 'by', 'Mike', 'To', '##bin', ',', 'a', 'reporter', 'for', 'the', 'network', ',', 'about', 'protests', 'in', 'Minnesota', 'and', 'elsewhere', '.', '[SEP]']


In [82]:
new_token,new_labels=[],[]
for token,label_idx in zip(tokens,label_indices[0]):
  if token.startswith("##"):
    #add at the end
    new_token[-1]=new_token[-1]+token[2:]
  else:
    #if its not special token
    new_labels.append(tag_values[label_idx])
    new_token.append(token) 

In [84]:
for token,label in zip(new_token,new_labels):
  print("{}\t{}".format(token,label))
  

[CLS]	O
Mr	O
.	O
Trump	O
’	O
s	O
tweets	O
began	O
just	O
moments	O
after	O
a	O
Fox	O
News	O
report	O
by	O
Mike	O
Tobin	O
,	O
a	O
reporter	O
for	O
the	O
network	O
,	O
about	O
protests	O
in	O
Minnesota	O
and	O
elsewhere	O
.	O
[SEP]	O


# For each batch of training data...